## Attention Mechanisms in Computer Vision: CBAM

For a more detailed breakdown of the Convolution Block Attention Module, check out the [full tutorial on the blog](https://blog.paperspace.com/attention-mechanisms-in-computer-vision-cbam/).

In [ ]:
# Model Parameter and FLOP counter
# !pip install --upgrade git+https://github.com/sovrasov/flops-counter.pytorch.git

In [2]:
!nvidia-smi

Tue Sep 29 16:44:52 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.36.06    Driver Version: 450.36.06    CUDA Version: 11.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Quadro P6000        On   | 00000000:00:05.0 Off |                  Off |
| 26%   49C    P0   149W / 250W |    987MiB / 24449MiB |     73%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [3]:
import time
import torch
import torch.nn as nn
from ptflops import get_model_complexity_info
import torch.nn.functional as F
import torch.nn.init as init
import os
import shutil
import time
import torch.nn.parallel
import torch.backends.cudnn as cudnn
import torch.optim
import torch.utils.data
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from torch.autograd import Variable
import numpy as np
import math
from torch.nn.parameter import Parameter

In [4]:
best_prec1 = 0
evaluate = True

In [5]:
def main():
    global best_prec1, evaluate

    __all__ = ['ResNet', 'resnet20', 'resnet32', 'resnet44', 'resnet56', 'resnet110', 'resnet1202']


    def _weights_init(m):
        classname = m.__class__.__name__
        #print(classname)
        if isinstance(m, nn.Linear) or isinstance(m, nn.Conv2d):
            init.kaiming_normal_(m.weight)


    class LambdaLayer(nn.Module):
        def __init__(self, lambd):
            super(LambdaLayer, self).__init__()
            self.lambd = lambd

        def forward(self, x):
            return self.lambd(x)


    class BasicConv(nn.Module):
        def __init__(self, in_planes, out_planes, kernel_size, stride=1, padding=0, dilation=1, groups=1, relu=True, bn=True, bias=False):
            super(BasicConv, self).__init__()
            self.out_channels = out_planes
            self.conv = nn.Conv2d(in_planes, out_planes, kernel_size=kernel_size, stride=stride, padding=padding, dilation=dilation, groups=groups, bias=bias)
            self.bn = nn.BatchNorm2d(out_planes,eps=1e-5, momentum=0.01, affine=True) if bn else None
            self.relu = nn.ReLU() if relu else None

        def forward(self, x):
            x = self.conv(x)
            if self.bn is not None:
                x = self.bn(x)
            if self.relu is not None:
                x = self.relu(x)
            return x

    class Flatten(nn.Module):
        def forward(self, x):
            return x.view(x.size(0), -1)

    class ChannelGate(nn.Module):
        def __init__(self, gate_channels, reduction_ratio=16, pool_types=['avg', 'max']):
            super(ChannelGate, self).__init__()
            self.gate_channels = gate_channels
            self.mlp = nn.Sequential(
                Flatten(),
                nn.Linear(gate_channels, gate_channels // reduction_ratio),
                nn.ReLU(),
                nn.Linear(gate_channels // reduction_ratio, gate_channels)
                )
            self.pool_types = pool_types
        def forward(self, x):
            channel_att_sum = None
            for pool_type in self.pool_types:
                if pool_type=='avg':
                    avg_pool = F.avg_pool2d( x, (x.size(2), x.size(3)), stride=(x.size(2), x.size(3)))
                    channel_att_raw = self.mlp( avg_pool )
                elif pool_type=='max':
                    max_pool = F.max_pool2d( x, (x.size(2), x.size(3)), stride=(x.size(2), x.size(3)))
                    channel_att_raw = self.mlp( max_pool )
                elif pool_type=='lp':
                    lp_pool = F.lp_pool2d( x, 2, (x.size(2), x.size(3)), stride=(x.size(2), x.size(3)))
                    channel_att_raw = self.mlp( lp_pool )
                elif pool_type=='lse':
                    # LSE pool only
                    lse_pool = logsumexp_2d(x)
                    channel_att_raw = self.mlp( lse_pool )

                if channel_att_sum is None:
                    channel_att_sum = channel_att_raw
                else:
                    channel_att_sum = channel_att_sum + channel_att_raw

            scale = torch.sigmoid( channel_att_sum ).unsqueeze(2).unsqueeze(3).expand_as(x)
            return x * scale

    def logsumexp_2d(tensor):
        tensor_flatten = tensor.view(tensor.size(0), tensor.size(1), -1)
        s, _ = torch.max(tensor_flatten, dim=2, keepdim=True)
        outputs = s + (tensor_flatten - s).exp().sum(dim=2, keepdim=True).log()
        return outputs

    class ChannelPool(nn.Module):
        def forward(self, x):
            return torch.cat( (torch.max(x,1)[0].unsqueeze(1), torch.mean(x,1).unsqueeze(1)), dim=1 )

    class SpatialGate(nn.Module):
        def __init__(self):
            super(SpatialGate, self).__init__()
            kernel_size = 7
            self.compress = ChannelPool()
            self.spatial = BasicConv(2, 1, kernel_size, stride=1, padding=(kernel_size-1) // 2, relu=False)
        def forward(self, x):
            x_compress = self.compress(x)
            x_out = self.spatial(x_compress)
            scale = torch.sigmoid(x_out) # broadcasting
            return x * scale

    class CBAM(nn.Module):
        def __init__(self, gate_channels, reduction_ratio=16, pool_types=['avg', 'max'], no_spatial=False):
            super(CBAM, self).__init__()
            self.ChannelGate = ChannelGate(gate_channels, reduction_ratio, pool_types)
            self.no_spatial=no_spatial
            if not no_spatial:
                self.SpatialGate = SpatialGate()
        def forward(self, x):
            x_out = self.ChannelGate(x)
            if not self.no_spatial:
                x_out = self.SpatialGate(x_out)
            return x_out


    class BasicBlock(nn.Module):
        expansion = 1

        def __init__(self, in_planes, planes, stride=1, option='A', use_cbam = True):
            super(BasicBlock, self).__init__()
            self.conv1 = nn.Conv2d(in_planes, planes, kernel_size=3, stride=stride, padding=1, bias=False)
            self.bn1 = nn.BatchNorm2d(planes)
            self.conv2 = nn.Conv2d(planes, planes, kernel_size=3, stride=1, padding=1, bias=False)
            self.bn2 = nn.BatchNorm2d(planes)
            self.use_cbam = use_cbam
            if self.use_cbam == True:
                self.cbam = CBAM(planes)

            self.shortcut = nn.Sequential()
            if stride != 1 or in_planes != planes:
                if option == 'A':
                    """
                    For CIFAR10 ResNet paper uses option A.
                    """
                    self.shortcut = LambdaLayer(lambda x:
                                                F.pad(x[:, :, ::2, ::2], (0, 0, 0, 0, planes//4, planes//4), "constant", 0))
                elif option == 'B':
                    self.shortcut = nn.Sequential(
                        nn.Conv2d(in_planes, self.expansion * planes, kernel_size=1, stride=stride, bias=False),
                        nn.BatchNorm2d(self.expansion * planes)
                    )
            

        def forward(self, x):
            out = F.relu(self.bn1(self.conv1(x)))
            out = self.bn2(self.conv2(out))
            out += self.shortcut(x)
            out = F.relu(out)
            if self.use_cbam == True:
                out = self.cbam(out)
            return out


    class ResNet(nn.Module):
        def __init__(self, block, num_blocks, num_classes=10):
            super(ResNet, self).__init__()
            self.in_planes = 16

            self.conv1 = nn.Conv2d(3, 16, kernel_size=3, stride=1, padding=1, bias=False)
            self.bn1 = nn.BatchNorm2d(16)
            self.layer1 = self._make_layer(block, 16, num_blocks[0], stride=1)
            self.layer2 = self._make_layer(block, 32, num_blocks[1], stride=2)
            self.layer3 = self._make_layer(block, 64, num_blocks[2], stride=2)
            self.linear = nn.Linear(64, num_classes)

            self.apply(_weights_init)

        def _make_layer(self, block, planes, num_blocks, stride):
            strides = [stride] + [1]*(num_blocks-1)
            layers = []
            for stride in strides:
                layers.append(block(self.in_planes, planes, stride))
                self.in_planes = planes * block.expansion

            return nn.Sequential(*layers)

        def forward(self, x):
            out = F.relu(self.bn1(self.conv1(x)))
            out = self.layer1(out)
            out = self.layer2(out)
            out = self.layer3(out)
            out = F.avg_pool2d(out, out.size()[3])
            out = out.view(out.size(0), -1)
            out = self.linear(out)
            return out


    def resnet20():
        return ResNet(BasicBlock, [3, 3, 3])


    def resnet32():
        return ResNet(BasicBlock, [5, 5, 5])


    def resnet44():
        return ResNet(BasicBlock, [7, 7, 7])


    def resnet56():
        return ResNet(BasicBlock, [9, 9, 9])


    def resnet110():
        return ResNet(BasicBlock, [18, 18, 18])


    def resnet1202():
        return ResNet(BasicBlock, [200, 200, 200])

    model = resnet20()

    with torch.cuda.device(0):
      flops, params = get_model_complexity_info(model, (3, 224, 224), as_strings=True, print_per_layer_stat=True)
      print('{:<30}  {:<8}'.format('Computational complexity: ', flops))
      print('{:<30}  {:<8}'.format('Number of parameters: ', params))

    model.cuda()

    cudnn.benchmark = True

    normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                     std=[0.229, 0.224, 0.225])

    train_loader = torch.utils.data.DataLoader(
        datasets.CIFAR10(root='./data', train=True, transform=transforms.Compose([
            transforms.RandomHorizontalFlip(),
            transforms.RandomCrop(32, 4),
            transforms.ToTensor(),
            normalize,
        ]), download=True),
        batch_size=128, shuffle=True,
        num_workers=4, pin_memory=True)

    val_loader = torch.utils.data.DataLoader(
        datasets.CIFAR10(root='./data', train=False, transform=transforms.Compose([
            transforms.ToTensor(),
            normalize,
        ])),
        batch_size=128, shuffle=False,
        num_workers=4, pin_memory=True)

    # define loss function (criterion) and pptimizer
    criterion = nn.CrossEntropyLoss().cuda()

    optimizer = torch.optim.SGD(model.parameters(), 0.1,
                                momentum=0.9,
                                weight_decay=5e-4)

    lr_scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer,
                                                        milestones=[100, 150], last_epoch=0 - 1)


    for epoch in range(0, 200):

        # train for one epoch
        print('current lr {:.5e}'.format(optimizer.param_groups[0]['lr']))
        train(train_loader, model, criterion, optimizer, epoch)
        lr_scheduler.step()

        # evaluate on validation set
        prec1 = validate(val_loader, model, criterion)

        # remember best prec@1 and save checkpoint
        is_best = prec1 > best_prec1
        best_prec1 = max(prec1, best_prec1)

        is_best = prec1 > best_prec1
        best_prec1 = max(prec1, best_prec1)
        if epoch > 0 and epoch % 20 == 0:
            save_checkpoint({
                'epoch': epoch + 1,
                'state_dict': model.state_dict(),
                'best_prec1': best_prec1,
            }, is_best, filename=os.path.join('./', 'vanilla_checkpoint.th'))

        save_checkpoint({
            'state_dict': model.state_dict(),
            'best_prec1': best_prec1,
        }, is_best, filename=os.path.join('./', 'vanilla_model.th'))



def train(train_loader, model, criterion, optimizer, epoch):
    """
        Run one train epoch
    """
    batch_time = AverageMeter()
    data_time = AverageMeter()
    losses = AverageMeter()
    top1 = AverageMeter()

    # switch to train mode
    model.train()

    end = time.time()
    for i, (input, target) in enumerate(train_loader):

        # measure data loading time
        data_time.update(time.time() - end)

        target = target.cuda()
        input_var = input.cuda()
        target_var = target

        # compute output
        output = model(input_var)
        loss = criterion(output, target_var)

        # compute gradient and do SGD step
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        output = output.float()
        loss = loss.float()
        # measure accuracy and record loss
        prec1 = accuracy(output.data, target)[0]
        losses.update(loss.item(), input.size(0))
        top1.update(prec1.item(), input.size(0))

        # measure elapsed time
        batch_time.update(time.time() - end)
        end = time.time()

        if i % 20 == 0:
            print('Epoch: [{0}][{1}/{2}]\t'
                  'Time {batch_time.val:.3f} ({batch_time.avg:.3f})\t'
                  'Data {data_time.val:.3f} ({data_time.avg:.3f})\t'
                  'Loss {loss.val:.4f} ({loss.avg:.4f})\t'
                  'Prec@1 {top1.val:.3f} ({top1.avg:.3f})'.format(
                      epoch, i, len(train_loader), batch_time=batch_time,
                      data_time=data_time, loss=losses, top1=top1))


def validate(val_loader, model, criterion):
    """
    Run evaluation
    """
    batch_time = AverageMeter()
    losses = AverageMeter()
    top1 = AverageMeter()

    # switch to evaluate mode
    model.eval()

    end = time.time()
    with torch.no_grad():
        for i, (input, target) in enumerate(val_loader):
            target = target.cuda()
            input_var = input.cuda()
            target_var = target.cuda()


            # compute output
            output = model(input_var)
            loss = criterion(output, target_var)

            output = output.float()
            loss = loss.float()

            # measure accuracy and record loss
            prec1 = accuracy(output.data, target)[0]
            losses.update(loss.item(), input.size(0))
            top1.update(prec1.item(), input.size(0))

            # measure elapsed time
            batch_time.update(time.time() - end)
            end = time.time()

            if i % 20 == 0:
                print('Test: [{0}/{1}]\t'
                      'Time {batch_time.val:.3f} ({batch_time.avg:.3f})\t'
                      'Loss {loss.val:.4f} ({loss.avg:.4f})\t'
                      'Prec@1 {top1.val:.3f} ({top1.avg:.3f})'.format(
                          i, len(val_loader), batch_time=batch_time, loss=losses,
                          top1=top1))

    print(' * Prec@1 {top1.avg:.3f}'
          .format(top1=top1))

    return top1.avg

def save_checkpoint(state, is_best, filename='checkpoint.pth.tar'):
    """
    Save the training model
    """
    torch.save(state, filename)

class AverageMeter(object):
    """Computes and stores the average and current value"""
    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count


def accuracy(output, target, topk=(1,)):
    """Computes the precision@k for the specified values of k"""
    maxk = max(topk)
    batch_size = target.size(0)

    _, pred = output.topk(maxk, 1, True, True)
    pred = pred.t()
    correct = pred.eq(target.view(1, -1).expand_as(pred))

    res = []
    for k in topk:
        correct_k = correct[:k].view(-1).float().sum(0)
        res.append(correct_k.mul_(100.0 / batch_size))
    return res



In [6]:
if __name__ == '__main__':
    main()

ResNet(
  0.273 M, 100.000% Params, 2.025 GMac, 100.000% MACs, 
  (conv1): Conv2d(0.0 M, 0.158% Params, 0.022 GMac, 1.070% MACs, 3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (bn1): BatchNorm2d(0.0 M, 0.012% Params, 0.002 GMac, 0.079% MACs, 16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (layer1): Sequential(
    0.014 M, 5.298% Params, 0.718 GMac, 35.469% MACs, 
    (0): BasicBlock(
      0.005 M, 1.766% Params, 0.239 GMac, 11.823% MACs, 
      (conv1): Conv2d(0.002 M, 0.844% Params, 0.116 GMac, 5.708% MACs, 16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(0.0 M, 0.012% Params, 0.002 GMac, 0.079% MACs, 16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(0.002 M, 0.844% Params, 0.116 GMac, 5.708% MACs, 16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(0.0 M, 0.012% Params, 0.002 GMac, 0.079% MACs, 16, eps=1e-0

Files already downloaded and verified
current lr 1.00000e-01
Epoch: [0][0/391]	Time 0.504 (0.504)	Data 0.271 (0.271)	Loss 2.3530 (2.3530)	Prec@1 11.719 (11.719)
Epoch: [0][20/391]	Time 0.050 (0.074)	Data 0.000 (0.013)	Loss 2.1574 (2.2528)	Prec@1 11.719 (15.179)
Epoch: [0][40/391]	Time 0.051 (0.063)	Data 0.000 (0.007)	Loss 2.1476 (2.1872)	Prec@1 18.750 (17.435)
Epoch: [0][60/391]	Time 0.042 (0.058)	Data 0.000 (0.005)	Loss 1.9859 (2.1469)	Prec@1 25.781 (19.121)
Epoch: [0][80/391]	Time 0.056 (0.058)	Data 0.000 (0.004)	Loss 2.0577 (2.1167)	Prec@1 20.312 (20.505)
Epoch: [0][100/391]	Time 0.051 (0.056)	Data 0.000 (0.003)	Loss 1.8887 (2.0841)	Prec@1 26.562 (21.651)
Epoch: [0][120/391]	Time 0.052 (0.055)	Data 0.000 (0.003)	Loss 1.8476 (2.0611)	Prec@1 34.375 (22.611)
Epoch: [0][140/391]	Time 0.061 (0.055)	Data 0.000 (0.002)	Loss 1.8697 (2.0312)	Prec@1 28.125 (23.947)
Epoch: [0][160/391]	Time 0.047 (0.055)	Data 0.000 (0.002)	Loss 1.8598 (2.0044)	Prec@1 33.594 (25.214)
Epoch: [0][180/391]	Time 0.

Epoch: [3][200/391]	Time 0.097 (0.099)	Data 0.000 (0.002)	Loss 1.0119 (1.0773)	Prec@1 65.625 (61.066)
Epoch: [3][220/391]	Time 0.103 (0.099)	Data 0.000 (0.002)	Loss 0.8922 (1.0764)	Prec@1 71.094 (61.174)
Epoch: [3][240/391]	Time 0.102 (0.099)	Data 0.000 (0.002)	Loss 1.2006 (1.0755)	Prec@1 57.812 (61.265)
Epoch: [3][260/391]	Time 0.094 (0.099)	Data 0.000 (0.001)	Loss 1.0936 (1.0736)	Prec@1 60.938 (61.386)
Epoch: [3][280/391]	Time 0.092 (0.099)	Data 0.000 (0.001)	Loss 0.9093 (1.0690)	Prec@1 67.188 (61.519)
Epoch: [3][300/391]	Time 0.090 (0.099)	Data 0.001 (0.001)	Loss 1.0149 (1.0659)	Prec@1 64.062 (61.737)
Epoch: [3][320/391]	Time 0.080 (0.098)	Data 0.001 (0.001)	Loss 1.0715 (1.0624)	Prec@1 64.062 (61.948)
Epoch: [3][340/391]	Time 0.073 (0.097)	Data 0.001 (0.001)	Loss 0.7559 (1.0579)	Prec@1 76.562 (62.154)
Epoch: [3][360/391]	Time 0.109 (0.097)	Data 0.000 (0.001)	Loss 0.9103 (1.0543)	Prec@1 72.656 (62.277)
Epoch: [3][380/391]	Time 0.094 (0.097)	Data 0.000 (0.001)	Loss 1.1792 (1.0524)	Pre

Test: [20/79]	Time 0.035 (0.056)	Loss 0.9305 (0.9601)	Prec@1 69.531 (68.899)
Test: [40/79]	Time 0.055 (0.051)	Loss 1.0636 (0.9598)	Prec@1 67.188 (69.150)
Test: [60/79]	Time 0.032 (0.051)	Loss 1.0263 (0.9572)	Prec@1 64.844 (68.929)
 * Prec@1 69.040
current lr 1.00000e-01
Epoch: [7][0/391]	Time 0.502 (0.502)	Data 0.365 (0.365)	Loss 0.6692 (0.6692)	Prec@1 75.781 (75.781)
Epoch: [7][20/391]	Time 0.086 (0.119)	Data 0.000 (0.018)	Loss 0.5989 (0.7151)	Prec@1 79.688 (76.116)
Epoch: [7][40/391]	Time 0.100 (0.106)	Data 0.000 (0.010)	Loss 0.7088 (0.7103)	Prec@1 76.562 (75.648)
Epoch: [7][60/391]	Time 0.101 (0.102)	Data 0.000 (0.007)	Loss 0.8169 (0.7167)	Prec@1 71.875 (75.205)
Epoch: [7][80/391]	Time 0.110 (0.102)	Data 0.000 (0.005)	Loss 0.5635 (0.7142)	Prec@1 79.688 (75.338)
Epoch: [7][100/391]	Time 0.100 (0.101)	Data 0.000 (0.004)	Loss 0.7087 (0.7127)	Prec@1 75.781 (75.549)
Epoch: [7][120/391]	Time 0.100 (0.101)	Data 0.000 (0.004)	Loss 0.6655 (0.7221)	Prec@1 78.125 (75.355)
Epoch: [7][140/391]	T

Epoch: [10][160/391]	Time 0.092 (0.103)	Data 0.000 (0.003)	Loss 0.5320 (0.6424)	Prec@1 81.250 (77.834)
Epoch: [10][180/391]	Time 0.092 (0.102)	Data 0.000 (0.003)	Loss 0.5627 (0.6436)	Prec@1 78.906 (77.728)
Epoch: [10][200/391]	Time 0.071 (0.101)	Data 0.000 (0.002)	Loss 0.7721 (0.6520)	Prec@1 73.438 (77.480)
Epoch: [10][220/391]	Time 0.092 (0.100)	Data 0.000 (0.002)	Loss 0.5903 (0.6532)	Prec@1 77.344 (77.422)
Epoch: [10][240/391]	Time 0.099 (0.100)	Data 0.000 (0.002)	Loss 0.7522 (0.6546)	Prec@1 77.344 (77.490)
Epoch: [10][260/391]	Time 0.078 (0.100)	Data 0.000 (0.002)	Loss 0.4460 (0.6550)	Prec@1 89.062 (77.481)
Epoch: [10][280/391]	Time 0.076 (0.101)	Data 0.000 (0.002)	Loss 0.6539 (0.6535)	Prec@1 78.906 (77.544)
Epoch: [10][300/391]	Time 0.095 (0.100)	Data 0.000 (0.002)	Loss 0.5432 (0.6513)	Prec@1 81.250 (77.624)
Epoch: [10][320/391]	Time 0.105 (0.100)	Data 0.000 (0.002)	Loss 0.7251 (0.6492)	Prec@1 79.688 (77.697)
Epoch: [10][340/391]	Time 0.105 (0.100)	Data 0.000 (0.001)	Loss 0.7552 (0

...

Test: [0/79]	Time 0.381 (0.381)	Loss 0.6801 (0.6801)	Prec@1 75.000 (75.000)
Test: [20/79]	Time 0.041 (0.056)	Loss 0.6833 (0.6590)	Prec@1 77.344 (77.418)
Test: [40/79]	Time 0.022 (0.043)	Loss 0.6074 (0.6669)	Prec@1 79.688 (77.687)
Test: [60/79]	Time 0.040 (0.038)	Loss 0.5155 (0.6643)	Prec@1 86.719 (78.074)
 * Prec@1 77.960
current lr 1.00000e-01
Epoch: [44][0/391]	Time 0.483 (0.483)	Data 0.378 (0.378)	Loss 0.5253 (0.5253)	Prec@1 80.469 (80.469)
Epoch: [44][20/391]	Time 0.105 (0.118)	Data 0.000 (0.018)	Loss 0.6392 (0.4676)	Prec@1 79.688 (83.668)
Epoch: [44][40/391]	Time 0.092 (0.109)	Data 0.000 (0.010)	Loss 0.5305 (0.4887)	Prec@1 85.156 (83.460)
Epoch: [44][60/391]	Time 0.101 (0.106)	Data 0.000 (0.007)	Loss 0.5161 (0.4926)	Prec@1 80.469 (83.120)
Epoch: [44][80/391]	Time 0.104 (0.105)	Data 0.009 (0.005)	Loss 0.5270 (0.4924)	Prec@1 80.469 (83.102)
Epoch: [44][100/391]	Time 0.102 (0.104)	Data 0.000 (0.004)	Loss 0.4340 (0.4938)	Prec@1 85.156 (83.014)
Epoch: [44][120/391]	Time 0.101 (0.103)	D

Epoch: [47][140/391]	Time 0.083 (0.099)	Data 0.002 (0.004)	Loss 0.6019 (0.4713)	Prec@1 81.250 (83.782)
Epoch: [47][160/391]	Time 0.099 (0.098)	Data 0.000 (0.003)	Loss 0.6144 (0.4764)	Prec@1 78.906 (83.662)
Epoch: [47][180/391]	Time 0.099 (0.098)	Data 0.000 (0.003)	Loss 0.4757 (0.4738)	Prec@1 85.156 (83.823)
Epoch: [47][200/391]	Time 0.100 (0.099)	Data 0.000 (0.003)	Loss 0.5767 (0.4813)	Prec@1 79.688 (83.567)
Epoch: [47][220/391]	Time 0.094 (0.099)	Data 0.001 (0.002)	Loss 0.4994 (0.4799)	Prec@1 82.031 (83.569)
Epoch: [47][240/391]	Time 0.092 (0.099)	Data 0.000 (0.002)	Loss 0.5673 (0.4804)	Prec@1 81.250 (83.548)
Epoch: [47][260/391]	Time 0.096 (0.099)	Data 0.000 (0.002)	Loss 0.3757 (0.4829)	Prec@1 90.625 (83.453)
Epoch: [47][280/391]	Time 0.101 (0.099)	Data 0.000 (0.002)	Loss 0.4371 (0.4837)	Prec@1 83.594 (83.399)
Epoch: [47][300/391]	Time 0.097 (0.099)	Data 0.000 (0.002)	Loss 0.5149 (0.4819)	Prec@1 80.469 (83.472)
Epoch: [47][320/391]	Time 0.094 (0.099)	Data 0.000 (0.002)	Loss 0.6710 (0

Epoch: [50][340/391]	Time 0.087 (0.097)	Data 0.001 (0.001)	Loss 0.4686 (0.4773)	Prec@1 84.375 (83.727)
Epoch: [50][360/391]	Time 0.103 (0.097)	Data 0.000 (0.001)	Loss 0.3880 (0.4772)	Prec@1 87.500 (83.708)
Epoch: [50][380/391]	Time 0.107 (0.097)	Data 0.000 (0.001)	Loss 0.4059 (0.4781)	Prec@1 83.594 (83.666)
Test: [0/79]	Time 0.336 (0.336)	Loss 0.8961 (0.8961)	Prec@1 72.656 (72.656)
Test: [20/79]	Time 0.057 (0.059)	Loss 0.7106 (0.9408)	Prec@1 75.781 (70.201)
Test: [40/79]	Time 0.053 (0.053)	Loss 1.1218 (0.9499)	Prec@1 66.406 (70.217)
Test: [60/79]	Time 0.049 (0.051)	Loss 0.9293 (0.9399)	Prec@1 71.094 (70.184)
 * Prec@1 70.240
current lr 1.00000e-01
Epoch: [51][0/391]	Time 0.396 (0.396)	Data 0.318 (0.318)	Loss 0.4901 (0.4901)	Prec@1 84.375 (84.375)
Epoch: [51][20/391]	Time 0.099 (0.114)	Data 0.000 (0.016)	Loss 0.7273 (0.4814)	Prec@1 73.438 (83.482)
Epoch: [51][40/391]	Time 0.099 (0.107)	Data 0.000 (0.008)	Loss 0.5156 (0.4828)	Prec@1 78.906 (83.270)
Epoch: [51][60/391]	Time 0.093 (0.104)	

Epoch: [54][80/391]	Time 0.103 (0.101)	Data 0.000 (0.004)	Loss 0.5393 (0.4550)	Prec@1 83.594 (84.568)
Epoch: [54][100/391]	Time 0.091 (0.101)	Data 0.000 (0.003)	Loss 0.3871 (0.4600)	Prec@1 85.156 (84.414)
Epoch: [54][120/391]	Time 0.087 (0.100)	Data 0.000 (0.003)	Loss 0.4752 (0.4606)	Prec@1 82.812 (84.407)
Epoch: [54][140/391]	Time 0.107 (0.100)	Data 0.000 (0.003)	Loss 0.5568 (0.4669)	Prec@1 78.906 (84.209)
Epoch: [54][160/391]	Time 0.115 (0.100)	Data 0.001 (0.002)	Loss 0.3299 (0.4663)	Prec@1 84.375 (84.191)
Epoch: [54][180/391]	Time 0.108 (0.101)	Data 0.000 (0.002)	Loss 0.4780 (0.4713)	Prec@1 86.719 (83.956)
Epoch: [54][200/391]	Time 0.091 (0.101)	Data 0.000 (0.002)	Loss 0.5040 (0.4773)	Prec@1 80.469 (83.808)
Epoch: [54][220/391]	Time 0.099 (0.101)	Data 0.001 (0.002)	Loss 0.3609 (0.4761)	Prec@1 86.719 (83.880)
Epoch: [54][240/391]	Time 0.092 (0.101)	Data 0.000 (0.002)	Loss 0.5274 (0.4735)	Prec@1 79.688 (83.931)
Epoch: [54][260/391]	Time 0.096 (0.100)	Data 0.000 (0.002)	Loss 0.4031 (0.

KeyboardInterrupt: 